<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Kleines-Beispiel" data-toc-modified-id="Kleines-Beispiel-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Kleines Beispiel</a></span></li></ul></div>

In [1]:
import sys
import os

try:
    os.chdir(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    sys.path.append(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    
import warnings
warnings.filterwarnings("ignore")

In [2]:
import nglview as nv
import MDAnalysis as mda
import numpy as np
import time
import ipywidgets as widgets
from scipy.sparse import csr_array
from numpy.typing import NDArray

from molgri.molecules.transitions import SimulationHistogram, MSM, SQRA
from molgri.plotting.molecule_plots import TrajectoryPlot
import pandas as pd
import matplotlib.pyplot as plt
from molgri.molecules.parsers import FileParser, ParsedEnergy, XVGParser

from molgri.paths import PATH_OUTPUT_PT, OUTPUT_PLOTTING_DATA, PATH_OUTPUT_LOGGING
from molgri.space.fullgrid import FullGrid
from molgri.space.utils import k_argmin_in_array, k_argmax_in_array

Erstmals: wir brauchen eine HF Monomer Datai.

In [4]:
%pycat input/HF.xyz

Nun lass zwei PTs (klein, groß) produzieren:
- klein: -o 42 -t "linspace(0.1, 0.4, 10)" -b 8
- groß: -o 162 -t "linspace(0.1, 0.4, 20)" -b 40

## Kleines Beispiel
- obtain full grid
- show full grid coordinates per frame in the corner

In [8]:
small_name = "HF_HF_0010"
b_grid = "8"
o_grid = "12"
t_grid = "linspace(0.2, 0.4, 3)"

In [4]:
%run molgri/scripts/generate_pt.py -m1 HF -m2 HF --recalculate -o $o_grid -b $b_grid -t "linspace(0.2, 0.4, 3)"

Saved the log file to output/data/logging/HF_HF_0010.log
Saved pseudo-trajectory to output/data/pt_files/HF_HF_0010.xtc and structure file to output/data/pt_files/HF_HF_0010.gro
Timing the generation of Pt HF_HF_0010: 0:00:00.479715 hours:minutes:seconds


In [110]:
class PTFrameAnalyser:
    
    """
    Visualise frames and important information from PT using NGLView. Reads PTs from saved files.
    """
    
    def __init__(self, u, fg=None):
        """
        Args:
            - name_pt: Name that is given to the PT files, usually in the form <molecule 1>_<molecule 2>_<4-number index>
                       or selected by the user
            - o_input: a string given as user input to create rotations around origin, e.g. "15" or "ico_15"
            - b_input: a string given as user input to create rotations around body, e.g. "42" or "cube4D_42"
            - t_input: a string given as user input to create translations, e.g. "[0.1, 0.2]" or "range(1, 5, 3)"
        """
        self.u = u
        self.fg = fg
        
        
                
    def _get_fullgrid_name(self):
        """
        NOT IN USE CURRENTLY
        Read from the .log file to obtain the name of the used FullGrid.
        """
        with open(f"{PATH_OUTPUT_LOGGING}{self.name_pt}.log") as f:
            while True:
                line = f.readline()
                if line.startswith("INFO:PtLogger:full grid name:"):
                    return line.strip().split(": ")[-1]
        
    
    def print_ith_information(self, i):
        """
        Print out important information about the i-th point of PT
        """
        all_points = self.fg.get_full_grid_as_array()
        ith_fg_coo = np.array2string(all_points[i], precision=2, suppress_small=True, separator=",")
        print(f"Frame {i}")
        print(f"FullGrid coordinates {ith_fg_coo}")
        
        neighbours = self._get_neig_indices(i)
        print(f"The point {i} has {len(neighbours)} neighbours, here is the list:")
        for n_index in neighbours:
            print(np.array2string(all_points[n_index], precision=2, suppress_small=True, separator=","))

def create_u_for_PT(name_pt: str, o_input: str, b_input: str, t_input: str):
    u = mda.Universe(f"{PATH_OUTPUT_PT}{self.name_pt}.gro", f"{PATH_OUTPUT_PT}{self.name_pt}.xtc")
    fg = FullGrid(o_grid_name=o_input, b_grid_name=b_input, t_grid_name=t_input, use_saved=True)
    _check_grid_correct(name_pt, fg)
    return u, fg

def _check_grid_correct(self, name_pt, fg):
    """
    Since we re-construct the FullGrid object to access all of its methods, we perform a quick check that the
    newly constructed grid actually has the same coordinates as the grid that has been saved during construction
    of the pseudotrajectory.
    """
    # first step: read the name of the full grid from a PT log file
    with open(f"{PATH_OUTPUT_LOGGING}{name_pt}.log") as f:
        while True:
            line = f.readline()
            if line.startswith("INFO:PtLogger:full grid name:"):
                full_grid_name = line.strip().split(": ")[-1]
                break

    # second step: load the .npy file with the found name
    used_grid = np.load(f"{OUTPUT_PLOTTING_DATA}get_full_grid_as_array_{full_grid_name}.npy")

    # third step: assert that this is actually the grid that has been used
    assert np.allclose(used_grid, fg.get_full_grid_as_array())
    



In [4]:
pta = PTFrameAnalyser(*create_u_for_PT(small_name, b_input=b_grid, o_input=o_grid, t_input=t_grid))
#view1 = pta.plot_ith_frame(15)
#view2 = pta.plot_ith_frame(75)
#pta.plot_side_by_side(view1, view2)
pta.print_ith_information(5)
pta.plot_all_neighours_of_i(5)

NameError: name 'small_name' is not defined

In [ ]:
view1 = pta.plot_ith_frame(15)
view2 = pta.plot_ith_frame(75)

In [ ]:
pta.plot_side_by_side(view1, view2)

- plot neighbours
- plot structures with largest phasespace volumes

In [6]:
# großeres beispiel
big_name = "HF_HF_0011"
b_grid_b = "272"
o_grid_b = "162"
t_grid_b = "linspace(0.1, 0.4, 20)"

In [4]:
%run molgri/scripts/generate_pt.py -m1 HF -m2 HF --recalculate -o $o_grid_b -b $b_grid_b -t "linspace(0.1, 0.4, 20)"

Saved the log file to output/data/logging/HF_HF_0011.log
Saved pseudo-trajectory to output/data/pt_files/HF_HF_0011.xtc and structure file to output/data/pt_files/HF_HF_0011.gro
Timing the generation of Pt HF_HF_0011: 0:24:59.522670 hours:minutes:seconds


In [7]:
pta = PTFrameAnalyser(big_name, b_input=b_grid_b, o_input=o_grid_b, t_input=t_grid_b)

pta.print_ith_information(500)
pta.plot_all_neighours_of_i(500)

TypeError: PTFrameAnalyser.__init__() got an unexpected keyword argument 'b_input'

In [3]:
%run molgri/scripts/generate_pt.py -m1 H2O -m2 H2O --recalculate -o "162" -b "272" -t "linspace(0.1, 0.4, 20)"

Saved the log file to output/data/logging/H2O_H2O_0092.log
Saved pseudo-trajectory to output/data/pt_files/H2O_H2O_0092.xtc and structure file to output/data/pt_files/H2O_H2O_0092.gro
Timing the generation of Pt H2O_H2O_0092: 0:25:25.037286 hours:minutes:seconds


In [ ]:
#WATER

In [27]:
from MDAnalysis import transformations

u = mda.Universe(r"/home/hanaz63/nobackup/gromacs/H2O_H2O_0095_10000/H2O_H2O_0095.gro", 
                 r"/home/hanaz63/nobackup/gromacs/H2O_H2O_0095_10000/output.xtc")

#reconnect molecules


ag = u.atoms
# we will use mass as weights for the center calculation
workflow = (transformations.center_in_box(ag, center='mass'),)
u.trajectory.add_transformations(*workflow)


v = nv.show_mdanalysis(u)
v.add_unitcell()
v

NGLWidget(max_frame=10000)

In [3]:
from molgri.molecules.parsers import XVGParser

        

my_parser = XVGParser(r"/home/hanaz63/nobackup/gromacs/H2O_H2O_0095_10000/full_energy.xvg")
potentials = my_parser.get_parsed_energy()

print(potentials.labels)


('None', 'LJ (SR)', 'Disper. corr.', 'Coulomb (SR)', 'Potential')


In [30]:
pta = PTFrameAnalyser(u)
pta.plot_ith_frame(min_frame)

NGLWidget()

In [37]:
my_parser = XVGParser(r"/home/hanaz63/nobackup/gromacs/H2O_H2O_0096/H2O_H2O_0096.xvg")
potentials = my_parser.get_parsed_energy().get_energies("Disper. corr.")
min_frame = np.argmin(potentials)
min_frame

432

In [5]:
%matplotlib notebook

In [183]:
class ViewManager:
    
    """
    NGLViewer is very useful but not very convinient for displaying particular frames of a trajectory together, 
    in particular color schemes etc. This class accepts a MDA Universe and knows how to extract frames from it.
    The plotting functions then accept indices (one or several) for this trajectory and display them in a 
    particular way (overlapping, sequential ...)
    """
    
    def __init__(self, u: mda.Universe):
        self.u = u
        self.fresh_view()
    
    def fresh_view(self):
        """
        Run this when you want to start a new view and discard an old one.
        """
        self.view = nv.NGLWidget()
    
    def get_ith_frame(self, i: int) -> mda.Universe:
        """
        The most important method acting on self.u. Get the Universe object containing of all the atoms 
        but only a single frame (i-th frame).
        
        Args:
            i (int): the index of the frame wanted
        
        Returns:
            a Universe containing the i-th frame of self.u
        """
        self.u.trajectory[i]
        all_atoms = self.u.select_atoms('all')
        new_u = mda.Merge(self.u.atoms)
        return new_u
    
    def _add_coordinate_axes(self):
        """
        Helper method. Add the x, y and z axes at origin to a NGLView.
        """
        
        # arguments of add_arrow are: start position, end position, color (in RGB), radius of arrow head
        # arguments of add_label are: position, color (in RGB), size, text
        
        # X-axis is red
        self.view.shape.add_arrow([0, 0, 0], [1, 0, 0],[ 1, 0, 0 ], 0.1)
        view.shape.add_label([1, 0, 0], [1, 0, 0], 1.5, 'x')
        
        # Y-axis is green
        self.view.shape.add_arrow([0, 0, 0], [0, 1, 0],[0, 1, 0 ], 0.1)
        self.view.shape.add_label([0, 1, 0], [0, 1, 0], 1.5, 'y')
        
        # Z-axis is blue
        self.view.shape.add_arrow([0, 0, 0], [0, 0, 1],[0, 0, 1 ], 0.1)
        self.view.shape.add_label([0, 0, 1], [0, 0, 1], 1.5, 'z')

    def plot_ith_frame(self, frame_i: int, axes: bool = True, **kwargs):
        """
        Plot i-th frame of self.u, adding to self.view.
        
        Args:
            - i: index of the frame
            - axes: if True, draw x, y and z axes
        """
        ith_atoms = self.get_ith_frame(frame_i)
        
        self.view.add_component(ith_atoms, default_representation=False)
        # the index is there in order to only affect the last added representation
        self.view[-1].add_representation("ball+stick", **kwargs)
        if axes:
            self._add_coordinate_axes()
        return self.view
    
    def _add_optional_representation_parameters(self, my_index: int, colors: list, opacities: list):
        """
        Helper method if you want to plot several view and pass arguments to them.
        """
        kwargs = {}
        if colors is not None:
            kwargs["color"] = colors[my_index]
        if opacities is not None:
            kwargs["opacity"] = opacities[my_index]
        return kwargs
        
    
    def plot_frames_sequential(self, list_indices: list, colors: list = None, opacities: list = None):
        """
        Plot several frames of the self.u next to each other. Automatically ngo to next now if you have too
        many frames to display in one row.
        
        Args:
            - list_indices: a list of integers, each an frame index to be displayed
            - colors: a list of colors (must be same length as list_indices) or None (default)
            - opacities: a list of opacities (must be same length as list_indices) or None (default)
        """
        
        # settings that are important so that rows with too many images nicely overflow in the next row
        box = widgets.Box(layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
        box.overflow_x = 'auto'

        all_views = []
        for li, list_i in enumerate(list_indices):
            self.fresh_view()
            # add optional parameters
            kwargs = self._add_optional_representation_parameters(li, colors, opacities)
            neig_view = self.plot_ith_frame(list_i, **kwargs)
            # this is also important for nice arragement of figures
            neig_view.layout.width = "200px"
            all_views.append(neig_view)
        
        
        # sync all views (so that all plots move if you move any)
        for v in all_views:
            v._set_sync_camera(all_views)
        
        box.children=[i for i in all_views]
        display(box)
    
    def plot_frames_overlapping(self, list_indices: list, colors: list = None, opacities: list = None):
        """
        Plot several frames of the self.u overlapping.
        
        Args:
            - list_indices: a list of integers, each an frame index to be displayed
            - colors: a list of colors (must be same length as list_indices) or None (default)
            - opacities: a list of opacities (must be same length as list_indices) or None (default)
        
        """
        
        for li, list_i in enumerate(list_indices):
            # add optional parameters
            kwargs = self._add_optional_representation_parameters(li, colors, opacities)
                
            self.plot_ith_frame(list_i, **kwargs)

        return self.view




In [ ]:
"""
Some important indices you may wanna plot:
- lowest energy structures
- neighbours
- all with same orientation
- all with same position
- eigenvectors of SQRA
"""

class PostCalculationEvaluator:
    
    """
    Combine the following (if you have it available):
        - trajectory or pseudotrajectory (in form of mda Universe)
        - emnergies
        - full grid (FullGrid object)
        - transition model(MSM or SQRA object)
        
    
    """
    
    def __init__(self, name_pt):
        self.name_pt = name_pt
        self.u = self.read_u_PT()
        self.parsed_trajectory = self.read_parsed_trajectory_PT()
        self.fg = self.read_fg_PT()
        self.transition_model = self.read_sqra_PT()
    
    def read_u_PT(self):
        return mda.Universe(f"{PATH_OUTPUT_PT}{self.name_pt}.gro", f"{PATH_OUTPUT_PT}{self.name_pt}.xtc")

    def read_parsed_trajectory_PT(self):
        pass

    def read_fg_PT(self):

        input_names = None
        full_grid_name = None

        # first step: read the name of the full grid from the log file
        with open(f"{PATH_OUTPUT_LOGGING}{self.name_pt}.log") as f:
            while input_names is None or full_grid_name is None:
                line = f.readline()
                if line.startswith("INFO:PtLogger:input grid parameters:"):
                    input_names = line.strip().split(": ")[-1]
                elif line.startswith("INFO:PtLogger:full grid name:"):
                    full_grid_name = line.strip().split(": ")[-1]
                    
        self.grid_name = full_grid_name
        
        input_names = input_names.split(" ")
        t_input = " ".join(input_names[2:])
        fg = FullGrid(o_grid_name=input_names[0], b_grid_name=input_names[1], t_grid_name=t_input,
                      use_saved=True)

        # second step: load the .npy file with the found name
        used_grid = np.load(f"{OUTPUT_PLOTTING_DATA}get_full_grid_as_array_{full_grid_name}.npy")

        # third step: assert that this is actually the grid that has been used
        assert np.allclose(used_grid, fg.get_full_grid_as_array())

        return fg

    def read_energy_PT(self):
        my_parser = XVGParser(f"/home/hanaz63/nobackup/gromacs/{self.name_pt}/{self.name_pt}.xvg")
        return my_parser.get_parsed_energy()

    def read_sqra_PT(self):
        sh = SimulationHistogram(self.parsed_trajectory, self.fg)
        return SQRA(sh, use_saved=True)
    
    
    
    def get_indices_k_lowest_energies(self, k: int, energy_type: str):
        pass
    
    def get_indices_neighbours_of_cell_i(self, i: int):
        adj_array = csr_array(self.fg.get_full_adjacency())[:, [i]].toarray().T[0]
        neighbour_indices = np.nonzero(adj_array)[0]
        return neighbour_indices

In [196]:
# EXAMPLE

# changeable parameters
my_name1 = "H2O"
my_name2 = "H2O"
my_num = "0099"
my_name = f"{my_name1}_{my_name2}_{my_num}"


# read everything from files 
def read_u_PT(name_pt):
    return mda.Universe(f"{PATH_OUTPUT_PT}{name_pt}.gro", f"{PATH_OUTPUT_PT}{name_pt}.xtc")

def read_parsed_trajectory_PT(name_pt):
    pass

def read_fg_PT(name_pt):

    input_names = None
    full_grid_name = None
    
    # first step: read the name of the full grid from the log file
    with open(f"{PATH_OUTPUT_LOGGING}{name_pt}.log") as f:
        while input_names is None or full_grid_name is None:
            line = f.readline()
            if line.startswith("INFO:PtLogger:input grid parameters:"):
                input_names = line.strip().split(": ")[-1]
            elif line.startswith("INFO:PtLogger:full grid name:"):
                full_grid_name = line.strip().split(": ")[-1]
    input_names = input_names.split(" ")
    t_input = " ".join(input_names[2:])
    fg = FullGrid(o_grid_name=input_names[0], b_grid_name=input_names[1], t_grid_name=t_input,
                  use_saved=True)
    
    # second step: load the .npy file with the found name
    used_grid = np.load(f"{OUTPUT_PLOTTING_DATA}get_full_grid_as_array_{full_grid_name}.npy")

    # third step: assert that this is actually the grid that has been used
    assert np.allclose(used_grid, fg.get_full_grid_as_array())
    
    return fg

def read_energy_PT(name_pt):
    my_parser = XVGParser(f"/home/hanaz63/nobackup/gromacs/{name_pt}/{name_pt}.xvg")
    return my_parser.get_parsed_energy()

def read_sqra_PT(name_pt, name_grid):
    SimulationHistogram()
    pass
    
print(read_u_PT(my_name))
print(read_fg_PT(my_name))
print(read_energy_PT(my_name))
    
# preparing the parsed trajectory
 #.get_energies("Disper. corr.")
pt_parser = FileParser(
    path_topology=f"/home/hanaz63/nobackup/gromacs/H2O_H2O_{my_num}/H2O_H2O_{my_num}.gro",
    path_trajectory=f"/home/hanaz63/nobackup/gromacs/H2O_H2O_{my_num}/H2O_H2O_{my_num}.xtc")
parsed_trajectory = pt_parser.get_parsed_trajectory(default_atom_selection="bynum 4:6")
parsed_trajectory.energies = pe


# example universe and fg
example_universe = pt_parser.universe


# display
vm = ViewManager(example_universe)
vm.fresh_view()
colors = ["red", "green", "blue"]
opacities = [0.5, 0.5, 0.5]
vm.plot_frames_sequential([5, 70, 200])


#print(pe.get_energies("Potential"))
# preparing the grid
#fg = FullGrid(t_grid_name="[0.1, 0.2, 0.3]", o_grid_name="ico_42", b_grid_name="cube4D_16")

#sh = SimulationHistogram(parsed_trajectory, fg)
#my_msm = MSM(sh, tau_array=np.array([10, 50, 100]), use_saved=False)

<Universe with 6 atoms>
[ 6.35023376e+02  5.95563599e+02  8.76023682e+02  4.06369202e+02
  9.78464600e+02  4.69440369e+02  5.05195984e+02  7.44091675e+02
  5.47452637e+02  3.93801086e+02  5.47453552e+02  5.63864197e+02
  5.63694763e+02  4.85781067e+02  3.93757812e+02  4.85815552e+02
  8.51343933e+02  5.61660339e+02  6.06557251e+02  7.93638184e+02
  1.13981360e+03  4.63972809e+02  9.07294678e+02  6.58014221e+02
  5.63553528e+02  4.62818573e+02  4.05159271e+02  5.44712036e+02
  5.29479187e+02  3.15578369e+02  3.38468872e+02  4.27176880e+02
  8.76219360e+02  5.05194458e+02  6.35103699e+02  9.78455811e+02
  4.06359924e+02  7.44108704e+02  5.95600708e+02  4.69474854e+02
  6.06474487e+02  9.07261230e+02  8.51097046e+02  1.13981152e+03
  7.93704651e+02  6.58120117e+02  5.61739685e+02  4.63926758e+02
  1.22662549e+03  5.32233582e+02  1.22664099e+03  8.14526672e+02
  8.14530457e+02  8.55376160e+02  5.32306458e+02  8.55387268e+02
  4.05206238e+02  3.38479736e+02  5.63511047e+02  5.29479492e+02
 

Box(children=(NGLWidget(layout=Layout(width='200px')), NGLWidget(layout=Layout(width='200px')), NGLWidget(layo…

In [111]:
pta = PTFrameAnalyser(pt_parser.universe)

# zeroth eigenvector
fg = FullGrid(o_grid_name="12", b_grid_name="8", t_grid_name="linspace(0.2, 0.5, 10)", use_saved=True)
#fg = FullGrid(o_grid_name="42", b_grid_name="40", t_grid_name="linspace(0.2, 0.5, 20)", use_saved=True)

sim_hist = SimulationHistogram(parsed_trajectory, full_grid=fg)

sqra = SQRA(sim_hist)
evalu, evec = sqra.get_eigenval_eigenvec()

my_eigenvector = evec[0].T[1]


import matplotlib

class MplColorHelper:

  def __init__(self):
    self.cmap = matplotlib.cm.get_cmap('bwr')
    self.norm = matplotlib.colors.TwoSlopeNorm(vcenter=0)
    self.scalarMap = matplotlib.cm.ScalarMappable(norm=self.norm, cmap=self.cmap)

  def get_rgb(self, val):
    return self.scalarMap.to_rgba(val)


helper = MplColorHelper()


rgba = np.array([helper.get_rgb(val) for val in my_eigenvector])

used_is = list(k_argmin_in_array(my_eigenvector, 15))
used_is.extend(list(k_argmax_in_array(evec[0].T[1], 15)))

colors = ["blue"]*15
colors.extend(["red"]*15)

view = None
for i, comp in enumerate(used_is):
    view = pta.plot_ith_frame(used_is, colors=colors[i], opacity=0.2, view=view, ij=i)
view

In 4D, only approximate calculation of Voronoi cells is possible. Proceeding numerically.


NGLWidget()

In [163]:
"""
HOW-TO: plotting different components in the same view with various properties
"""

used_is = list(k_argmin_in_array(my_eigenvector, 5))
used_is.extend(list(k_argmax_in_array(my_eigenvector, 5)))


colors = ["blue"]*5
colors.extend(["red"]*5)

view = nv.NGLWidget()
for i, comp in enumerate(used_is):
    view.add_component(pta.get_ith_frame(comp), default_representation=False)
    view[i].add_representation("ball+stick", color=colors[i], opacity=0.2)
view

NGLWidget()

In [17]:
energ = pe.get_energies("Potential")
smallest_i = k_argmin_in_array(energ, 20)
print("Smallest indices:", smallest_i)
print("Smallest energies:", energ[smallest_i])

print(np.argmin(pe.get_energies("Potential")))
print(pe.get_energies("Potential")[9907])
print(pe.get_energies("LJ (SR)")[9907])
print(pe.get_energies("Coulomb (SR)")[9907])


Smallest indices: [ 9907  9421  7861  9541  8499 11221  7261  8941  8227 11587 11585  9888
  8208 11568  9905 10621  8619 11101  8653  7741]
Smallest energies: [-26.530365 -25.242062 -23.816832 -25.970745 -23.493708 -24.348461
 -23.811672 -25.971127 -24.993198 -24.678345 -23.690573 -26.53035
 -24.993174 -24.678284 -25.250351 -24.346889 -23.492529 -23.687481
 -23.432158 -23.270643]
9907
-26.530365
5.883503
-34.884674


In [18]:
u=mda.Universe(f"/home/hanaz63/nobackup/gromacs/H2O_H2O_{my_num}/H2O_H2O_{my_num}.gro",
                                    f"/home/hanaz63/nobackup/gromacs/H2O_H2O_{my_num}/H2O_H2O_{my_num}.xtc")
pta = PTFrameAnalyser(u)
pta.plot_frames(smallest_i)

In [5]:
from ipyleaflet import WidgetControl, Map
from ipywidget import Text

m = Map(center=(18, 77), zoom=5)
address_box = Text(description = 'Enter Address')

widget_control = WidgetControl(widget=address_box, position='topright')
m.add_control(widget_control)


ModuleNotFoundError: No module named 'ipywidget'